In [10]:
from google.colab import drive
import os
import pandas as pd

current_directory = "/data"
os.chdir(current_directory)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Concatenate Youtube Data Information With Aspects

In [ ]:
#Load raw data
import json

with open('Jan23.json', 'r') as f:
  jan_raw_data = json.load(f)

with open('Feb23.json', 'r') as f:
  feb_raw_data = json.load(f)

with open('demo_absa.json', 'r') as f:
  absa_data = json.load(f)

#Filter to videos kept for demo
video_id_lst = [video_id for video_id, review_dict in absa_data.items()]
raw_data = dict(jan_raw_data,**feb_raw_data)
raw_data = dict(filter(lambda pair: pair[0] in video_id_lst, raw_data.items()))

delete_lst = ["description","publish time","transcript_raw","transcript_text","first_page_comments_list","chapters"]
for video_id, video_dict in raw_data.items():
  #Delete irrelevant data
  for key in delete_lst:
    del video_dict[key]
  absa_dict = absa_data[video_id]
  #Add video details to each product as we will use product as key instead of video
  for product, product_dict in absa_dict.items():
    product_dict["Video_Details"] = video_dict
absa_data

{'FymRBl76Pnw': {'UKISS Volume Booster Lash Primer Mascara': {'original': 'It is cheap and easy to use\n, this pink one from UKISS Eyelash primer\nIt is a\nfan-shaped brush and it is\nalso black. after you apply it, it feels like a\nvery light mascara,\nbut it can keep your curls lasting all day long, and it\nwill not faint or fall off.I feel It is\nbetter\nthan the Aidu yarn I used before.It\nmakes your eyelashes distinct.I\ncan feel at ease when I go\nout with it, especially when\n \nI go out to play.',
   'aspects': [{'Aspect': 'Price',
     'Opinion': 'Positive',
     'Explanation': 'It is cheap'},
    {'Aspect': 'Ease of use',
     'Opinion': 'Positive',
     'Explanation': 'Easy to use'},
    {'Aspect': 'Texture',
     'Opinion': 'Positive',
     'Explanation': 'it feels like a very light mascara'},
    {'Aspect': 'Longevity',
     'Opinion': 'Positive',
     'Explanation': 'it can keep your curls lasting all day long and it will not faint or fall off. I\ncan feel at ease when I 

# Group Reviews on the same product together

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 45.2 MB/s eta 0:00:00


In [ ]:
product_name_lst = []
for video_id in video_id_lst:
  absa_dict = absa_data[video_id]
  for product, product_dict in absa_dict.items():
    product_name_lst.append((video_id, product))

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

product_group_lst = []
visited = [0] * len(product_name_lst)
for i in range(len(product_name_lst)-1):
  video_id_product_pair = product_name_lst[i]
  product = video_id_product_pair[1]
  product_group = [video_id_product_pair]
  for j in range(i+1,len(product_name_lst)):
    if visited[j] == 0:
      other_video_id_product_pair = product_name_lst[j]
      other_product = other_video_id_product_pair[1]
      similarity = fuzz.WRatio(product, other_product)
      if similarity >= 90:
        visited[j] = 1
        product_group.append(other_video_id_product_pair)
  product_group_lst.append(product_group)
  if len(product_group) > 1:
    print(product_group)

product_absa_dict = {}
for product_group in product_group_lst:
  main_product_name = product_group[0][1]
  product_absa_dict[main_product_name] = {}
  for video_id_product_pair in product_group:
    video_id = video_id_product_pair[0]
    org_product_name = video_id_product_pair[1]
    product_absa_dict[main_product_name][video_id] = absa_data[video_id][org_product_name]

product_absa_dict['Dior Backstage Rosy Glow blush']

[('IDyIdB3Bsv0', 'Dior Backstage Rosy Glow blush'), ('l65WunGik1c', 'Dior BACKSTAGE Rosy Glow Blush')]


{'IDyIdB3Bsv0': {'original': "And at a first pass, you can see that it is  pretty transparent. Even though the color in  the pan can be a little intimidating, it looks  like a bright pink. The pigment is definitely  buildable and it starts off pretty sheer. I feel like it gives a really pretty, not  exactly natural, but it gives a nice blush.  I have oily skin so I prefer powder blushes.  They just work better with my skin. Looking  at this blush in terms of packaging it is a  little disappointing. Although it's not super  lightweight, there's not a heavy weight that's  kind of expected from luxury brands. Also it's  super small. Tiny product and basically fits  right in the palm of my hand. I know blush  can take a while to get through even if you're  wearing it on a daily basis, it still could take  years to finish. But for the price that's pretty  disappointing. The powder is definitely smooth  and looks really good on the skin. There's also  a light floral scent. I really only smel

# Product Category Tagging

In [ ]:
from ProductTagger import ProductTagger
pt = ProductTagger()
product_categories, product_categories_lexicon = pt.get_product_categories_and_lexicon("product_category_lexicon.json")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
for product, product_dict in product_absa_dict.items():
  category_dict = pt.categorize_product(product, product_categories, product_categories_lexicon)
  category_list = [key for key,count in category_dict['identified_product_categories'].items()]
  print(product)
  print(category_list)
  product_dict["Categories"] = category_list

UKISS Volume Booster Lash Primer Mascara
['primer', 'mascara']
Make Up For Ever Skin Powder Foundation
['foundation', 'powder']
Catkin Summer Palace Light Loose Powder
['powder']
CLINIQUE Even Better All-Over Concealer
['concealer']
Jane Iredale Flocked Sponge Makeup Blender
['sponge']
Physicians Formula Mineral Wear Talc-Free Correcting Powder
['powder']
Physicians Formula Murumuru Butter Bronzer Palette
['bronzer', 'makeup palette']
Bobbi Brown Long-Wear Cream Eyeshadow Stick
['eyeshadow']
NYX SLIM EYE PENCIL - Dark Brown
['eyebrows']
Pure Desire Lip Gloss
['lip gloss']
PAT McGRATH LABS LUST Lip Gloss
['lip gloss']
Tarte Ultra Creamy Shape Tape Concealer
['concealer']
Supergoop! Unseen Sunscreen
['sunscreen']
Supergoop! Glowscreen Broad Spectrum Sunscreen Primer
['primer', 'sunscreen']
Sephora Collection Brightening Eye Cream
['eye cream']
Charlotte Tilbury Airbrush Flawless Setting Spray
['setting spray']
Elf Color Corrector
['corrector']
Nyx Blur Blur Foundation
['foundation']
Esse

# Output the Data

In [ ]:
with open("Final_Demo_ABSA.json", "w") as write_file:
  json.dump(product_absa_dict, write_file, indent=4)

# Brand Tagging

Brand lexicon created with: 

https://www.kaggle.com/datasets/kingabzpro/cosmetics-datasets

https://www.kaggle.com/datasets/mfsoftworks/cosmetic-products

https://www.kaggle.com/datasets/jithinanievarghese/nykaa-popular-brands-cosmetics-beauty-products

https://www.kaggle.com/datasets/raghadalharbi/all-products-available-on-sephora-website

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [53]:
import re
import pandas as pd
import json

with open('Final_Demo_ABSA.json', 'r') as f:
  data = json.load(f)

In [54]:
brand_lexicon = pd.read_csv("brand_lexicon.csv", encoding="latin1")["BRAND"]
brand_lexicon

0         BUTTER LONDON
1           BOBBI BROWN
2                INGLOT
3      MAKE UP FOR EVER
4          BAREMINERALS
             ...       
729            AXIOLOGY
730           L.A. GIRL
731            ONE SIZE
732                ILIA
733     DANESSA MYRICKS
Name: BRAND, Length: 734, dtype: object

In [39]:
product_lst = [product for product, product_dict in data.items()]

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def get_brand_from_product(product, brand_lexicon):
  for brand in brand_lexicon:
    if brand in product.upper():
      return brand

product_brand_lst = []
for product in product_lst:
  brand = get_brand_from_product(product, brand_lexicon)
  data[product]["brand"] = brand
data

# Engagement Rate Calculation

In [56]:
def get_engagement_rate(video_dict):
  like_count = int(video_dict["like count"])
  view_count = int(video_dict["view count"])
  comment_count = int(video_dict["comment count"])
  if view_count == 0:
    return 0
  return ((like_count + comment_count) / view_count) * 100

In [60]:
for product, product_dict in data.items():
  for key, item in product_dict.items():
    if type(item) == dict:
      if "Video_Details" in item:
        video_dict = product_dict[key]["Video_Details"]
        engagement_rate = get_engagement_rate(video_dict)
        video_dict["engagement rate"] = engagement_rate


In [63]:
with open("Final_Demo_ABSA_With_Brands.json", "w") as write_file:
  json.dump(data, write_file, indent=4)